# Group Recommender Systems - Tutorial 2 (Lab 2)

In this tutorial, we will focus on Group Recommender Systems. After completing this tutorial, you will be able to: 
- Implement the basic agregation strategies for group recommendations.
- Generate a simple textual explanation for such strategies.

#### Summary

1. Selection of a random group in our dataset
2. Aggregation Strategies for Group Recommenders
3. Explanations for Group Recommenders


#### 1. Selection of a random group in our dataset

First, we need a group! So, we will select a random set of five users from our dataset. For simplicity, we will focus on users having at least 200 evaluations.

In [2]:
preprocessed_dataset_folder = "../preprocessed_dataset"

import pandas as pd
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv") 
movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv", index_col="item")


In [3]:
users_ratings = ratings_df.groupby(['user']).count()
selected = users_ratings['rating'] > 200
selected_users = users_ratings.loc[selected]
random_selected = selected_users.sample(n=5) # sample() returns now n random rows from the dataframe. The returned object is a dataframe with five rows. 
select_column_df = random_selected.reset_index()['user'] # reset_index() create a new index, and the userId became a column. Then, we can filter using the column name
group_users = list(select_column_df) # iloc select by index, since our dataframe only has one row we read it from the index 0
print(group_users)

[573, 474, 596, 425, 480]


Let us assume we want to recommend to this group a list of 10 movies that nobody in the group has seen yet. We first need to determine the list of possible candidates. For simplicity, we will only consider movies for which we more then 10 evaluations.

In [5]:
group_ratings = ratings_df.loc[ratings_df['user'].isin(group_users)]
all_movies = set(movies_df.index.tolist())
num_ratings_df = ratings_df.groupby(['item']).count()
considered_movies = set(num_ratings_df.loc[num_ratings_df['user'] > 10].reset_index()['item'])

group_seen_movies = set(group_ratings['item'].tolist())
group_unseen_movies = considered_movies - group_seen_movies

print(len(all_movies))
print(len(considered_movies))
print(len(group_seen_movies))
print(len(group_unseen_movies))

4787
1450
1669
455


Now, we need to evaluate individuals' preverences for the unseen movies. To do so, we use the Lenskit library. We will use the same CF recommender used in the previous example. To generate the Dataframe with user-item pairs to pass as input in the *predict* function, we use the [product](https://docs.python.org/3/library/itertools.html#itertools.product) method of the itertools library, which takes as imput two lists and returns all the possible combinations between elements of the two lists. This is passed as input for the Dataframe constructor, which will then generate a Dataframe containing a pair on each row.

In [6]:
import itertools
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(ratings_df)
group_unseen_df = pd.DataFrame(list(itertools.product(group_users, group_unseen_movies)), columns=['user', 'item'])
group_unseen_df['predicted_rating'] = recsys.predict(group_unseen_df)
display(group_unseen_df)

,user,item,predicted_rating
0,573,2052,3.686284
1,573,2053,3.130455
2,573,9,3.583190
3,573,12,3.430130
4,573,2060,4.502876
...,...,...,...
2270,480,55276,3.680670
2271,480,112623,3.719628
2272,480,55282,3.031934
2273,480,55290,3.839130


We have now our predicted ratings.
We can apply an aggregation strategy to generate the group recommendations.

#### 2. Aggregation Strategies for Group Recommenders

Let's implement some of the aggregation strategies seen in the lecture today.

##### Additive strategy

The Additive strategy considers as group rating the sum of all the individuals ratings. Then, the recommended items are the one scoring the best with such group rating. We can easily implement it grouping our *group_unseen_df* Dataframe by *item*, and then computing the *sum*.

In [7]:
# Additive strategy

additive_df = group_unseen_df.groupby('item').sum()
additive_df = additive_df.join(movies_df['title'], on='item')
additive_df = additive_df.sort_values(by="predicted_rating", ascending=False).reset_index()[['item', 'title', 'predicted_rating']]
display(additive_df.head(10))

,item,title,predicted_rating
0,3328,ghost dog: the way of the samurai,21.514811
1,7387,dawn of the dead,21.405591
2,56782,there will be blood,21.368789
3,3740,big trouble in little china,21.338536
4,1273,down by law,21.229105
5,2921,high plains drifter,21.198174
6,94959,moonrise kingdom,21.153093
7,63082,slumdog millionaire,21.050718
8,1227,once upon a time in america,21.050584
9,866,bound,20.908362


##### Least Misery strategy

The Least Misery strategy considers as group rating the minimum of all the individuals ratings. Then, the recommended items are the one scoring the best with such group rating. As we did before, we can implement it grouping our *group_unseen_df* Dataframe by *item*, and then computing the *min*.

In [8]:
# least misery

least_misery_df = group_unseen_df.groupby('item').min()
least_misery_df = least_misery_df.join(movies_df['title'], on='item')
least_misery_df = least_misery_df.sort_values(by="predicted_rating", ascending=False).reset_index()[['item', 'title', 'predicted_rating']]
display(least_misery_df.head(10))

,item,title,predicted_rating
0,94959,moonrise kingdom,4.054671
1,7387,dawn of the dead,4.010506
2,3328,ghost dog: the way of the samurai,3.966930
3,63082,slumdog millionaire,3.956629
4,112175,how to train your dragon 2,3.948646
5,57640,hellboy ii: the golden army,3.909793
6,106100,dallas buyers club,3.902540
7,106920,her,3.896588
8,1273,down by law,3.896406
9,1287,ben-hur,3.895427


##### Most Pleasure strategy

The Most Pleasure strategy considers as group rating the maximum of all the individuals ratings. Then, the recommended items are the one scoring the best with such group rating. Again, We can easily implement it grouping our *group_unseen_df* Dataframe by *item*, and then computing the *max*.

In [9]:
# most pleasure

most_pleasure_df = group_unseen_df.groupby('item').max()
most_pleasure_df = most_pleasure_df.join(movies_df['title'], on='item')
most_pleasure_df = most_pleasure_df.sort_values(by="predicted_rating", ascending=False).reset_index()[['item', 'title', 'predicted_rating']]
display(most_pleasure_df.head(10))

,item,title,predicted_rating
0,3740,big trouble in little china,5.127776
1,1273,down by law,5.069009
2,56782,there will be blood,5.013777
3,2921,high plains drifter,5.002315
4,7387,dawn of the dead,4.970301
5,3328,ghost dog: the way of the samurai,4.936035
6,54190,across the universe,4.919382
7,1845,zero effect,4.919229
8,81847,tangled,4.887490
9,3783,croupier,4.882427


##### Fairness strategy

For the Fairness strategy we have an ordering between the group members, and at each round one group member choose the best item for him/her. Hence, we can compute the preference lists for each group member separately. Then we iterate over the group members, and at each iteration we select one element from the list of the correct user, and add it to the result list. Finally, we create a dataframe and enrich the information of the movies selected 

In [10]:
# Fairness

import pandas as pd

def generate_preference_list(user):
    individual_df = group_unseen_df.loc[group_unseen_df['user']==user]
    return list(individual_df.sort_values(by="predicted_rating", ascending=False).reset_index()['item'])

individual_preference_lists = dict()
for member in group_users:
    individual_preference_lists[member] = generate_preference_list(member)
    
result = list()
for i in range(10):
    user = group_users[i % 5]
    user_best = individual_preference_lists[user].pop(0)
    for member in group_users:
        if user_best in individual_preference_lists[member]:
            individual_preference_lists[member].remove(user_best)
    result.append(user_best)
    
fairness_df = pd.DataFrame(result, columns=['item']).join(movies_df['title'], on='item')
display(fairness_df)

,item,title
0,3740,big trouble in little china
1,174055,dunkirk
2,56782,there will be blood
3,3328,ghost dog: the way of the samurai
4,55820,no country for old men
5,1273,down by law
6,7387,dawn of the dead
7,1227,once upon a time in america
8,2921,high plains drifter
9,94959,moonrise kingdom


In [12]:
# To check individual evaluations on a specific item
group_unseen_df.loc[group_unseen_df['item']==3740]

,user,item,predicted_rating
390,573,3740,5.127776
845,474,3740,3.807201
1300,596,3740,4.276477
1755,425,3740,4.131123
2210,480,3740,3.995958


##### EXERCISE

Implement the Approval Voting and Plurality Voing strategies for group recommendations.

#### 2 Explanations for Group Recommenders

Let's see now some simple strategy to generate basic explanations for the group recommendation strategies implemented before. For the Additive, Least Misery and Most Pleasure strategies, we will use social-choice based explanations as defined in [Barile et al., 2021](http://ceur-ws.org/Vol-2955/paper11.pdf). For the Fairness strategy, we will use a generic formulation:

- Additive: "i_k has been recommended to the group since it achieves the highest total rating."
- Least Misery: "i_k has been recommended to the group since no group members has a real problem with it."
- Most Pleasure: "i_k has been recommended to the group since it achieves the highest of all individual group members."
- Fairness: "i_k has been recommended to the group since it is the favourite for u_j, and it's his/her turn to choose."

In [13]:
explanations = {
    "ADD" : "<item> has been recommended to the group since it achieves the highest total rating.\n",
    "LMS" : "<item> has been recommended to the group since no group members has a real problem with it.\n",
    "MPL" : "<item> has been recommended to the group since it achieves the highest of all individual group members.\n",
    "FAI" : "<item> has been recommended to the group since it is the favourite for <user>, and it's his/her turn to choose.\n"
}

# Present explanations for the first item of each strategy
movie_title = additive_df['title'].iloc[0]
print("Recommendation: " + movie_title.title())
print(explanations["ADD"].replace("<item>", "The movie \"" + movie_title.title() + "\""))

movie_title = least_misery_df['title'].iloc[0]
print("Recommendation: " + movie_title.title())
print(explanations["LMS"].replace("<item>", "The movie \"" + movie_title.title() + "\""))

movie_title = most_pleasure_df['title'].iloc[0]
print("Recommendation: " + movie_title.title())
print(explanations["MPL"].replace("<item>", "The movie \"" + movie_title.title() + "\""))

movie_title = fairness_df['title'].iloc[0]
user = group_users[0]
print("Recommendation: " + movie_title.title())
print(explanations["FAI"]
      .replace("<item>", "The movie \"" + movie_title.title() + "\"")
      .replace("<user>", "the user with id " + str(user)))

Recommendation: Ghost Dog: The Way Of The Samurai
The movie "Ghost Dog: The Way Of The Samurai" has been recommended to the group since it achieves the highest total rating.

Recommendation: Moonrise Kingdom
The movie "Moonrise Kingdom" has been recommended to the group since no group members has a real problem with it.

Recommendation: Big Trouble In Little China
The movie "Big Trouble In Little China" has been recommended to the group since it achieves the highest of all individual group members.

Recommendation: Big Trouble In Little China
The movie "Big Trouble In Little China" has been recommended to the group since it is the favourite for the user with id 573, and it's his/her turn to choose.



##### EXERCISE

Implement the explanation for the Approval Voting and Plurality Voting strategies for group recommendations, and print the corresponding explanation for the best movie for the group.